Install neccessary packages 

In [1]:
%pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from urllib import request as rq
import pandas as pd
import os
import pyarrow as pa # this is needed for the parquet file
import numpy as np
import ipywidgets
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, VBox
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder

In [2]:
if os.path.exists('./Data'):
    print('Data folder exists')

if os.path.exists('./Data/Bus'):
    print('Bus Data folder exists')
else:
    os.makedirs('./Data/Bus')

if os.path.exists('./Data/Weather/'):
    print('Weather data folder exists')
else:
    os.makedirs('./Data/Weather/')

if os.path.exists('./Zips/MetEirrean'):
    print('Weather folder exisits')
else:
    os.makedirs('./Zips/MetEirrean/')



Data folder exists
Bus Data folder exists
Weather folder exisits


Download the Dublin Bus data from https://data.gov.ie/organization/dublin-city-council?tags=Transport+and+Infrastructure

Use a generator to load the data into pandas data frame https://pandasninja.com/2019/04/how-to-read-lots-of-csv-files-easily-into-pandas/#:~:text=How%20to%20read%20lots%20of%20csv%20files%20easily,need%20...%204%204.%20Leverage%20regular%20expressions%20


In [3]:
# Function to load the Dublin bus gz files 
def load_Files(direc, files, comtype):
    columns = ['Timestamp', 'LineID', 'Direction', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Operator', 'Congestion', 'LonWGS84', 'LatWGS84', 'Delay', 'BlockID', 'VehicleID', 'StopID', 'AtStop']
    for f in files:
        print(f)
        yield pd.read_csv(direc + f, compression=comtype, delimiter=',', header=0, names=columns, parse_dates=True, low_memory=True)

def crackit_open(busFile):
    import zipfile as zip
    # Zip creates its own folders - no need to check for folder existence
    with zip.ZipFile(busFile,  mode='r') as arc: 
        arc.extractall('./Data/Bus/Gz/')  
    files = os.listdir('./Data/Bus/Gz/')
    DBfiles = [f for f in files if f.endswith('.gz')]
    df = pd.concat(load_Files('./Data/Bus/Gz/', DBfiles, 'gzip'), copy = False)
    return df

In [4]:

# Check parquet file existence before downloading - iof starting from afresh this takes a long time
if os.path.exists('./Data/Bus/CleanedBusData.parquet'):
    print("Parquest file exists, means the Bus data has been downloaded already ")
    df = pd.read_parquet('./Data/Bus/CleanedBusData.parquet')
elif os.path.exists('./Zips/Bus/DublinBusdata.zip'):
    print("Zip file exists, we have already downloaded the Dublin Bus Zip data, crack it open")
    df = crackit_open('./Zips/Bus/DublinBusdata.zip')
else:
    os.makedirs('./Zips/Bus/', exist_ok = True)
    url = "https://opendata.dublincity.ie/TrafficOpenData/sir010113-310113.zip"
    busFile = rq.urlretrieve(url, './Zips/Bus/DublinBusdata.zip' )  
    df = crackit_open('./Zips/Bus/DublinBusdata.zip')



Parquest file exists, means the Bus data has been downloaded already 


In [5]:
ColumnsToExclude = ('JourneyPatternID', 'StopID')

# Data is loaded - now needs to be cleaned, there is an argument to be made to include this in the above cell 
# but I have added it here so that I can add more columns to be excluded as I go rather than load the raw data each time
for c in ColumnsToExclude:
    if c in df.columns:  # type: ignore
        df = df.drop(c, axis=1)  # type: ignore
    else:
        print('Columns already dropped')




Columns already dropped
Columns already dropped


In [6]:
df.head()

,Timestamp,LineID,Direction,TimeFrame,VehicleJourneyID,Operator,Congestion,LonWGS84,LatWGS84,Delay,BlockID,VehicleID,AtStop,datetime,Time,Day,Hour,Minute
0,1356998405000000,27.0,0,2012-12-31,3883,RD,0,-6.233417,53.342232,0,27017,33521,0,2013-01-01 00:00:05,00:00:05,1,0,0
1,1356998407000000,40.0,0,2012-12-31,2226,HN,0,-6.278250,53.416683,0,40206,33142,0,2013-01-01 00:00:07,00:00:07,1,0,0
2,1356998407000000,7.0,0,2012-12-31,6106,D1,0,-6.231633,53.317768,0,7019,43004,1,2013-01-01 00:00:07,00:00:07,1,0,0
3,1356998411000000,747.0,0,2012-12-31,3531,SL,0,-6.254617,53.355484,-454,747007,40039,0,2013-01-01 00:00:11,00:00:11,1,0,0
4,1356998411000000,56.0,0,2012-12-31,1830,RD,0,-6.233183,53.342201,0,56001,33488,0,2013-01-01 00:00:11,00:00:11,1,0,0


In [7]:
# Deal with the timestamp and make it human readable
df['datetime'] = pd.to_datetime(df['Timestamp'], unit='us')
df['Time'] = df['datetime'].dt.time
df['Day'] = df['datetime'].dt.day
df['Hour'] = df['datetime'].dt.hour
df['Minute'] = df['datetime'].dt.minute

In [8]:
print(df.head())

          Timestamp  LineID  Direction   TimeFrame  VehicleJourneyID Operator  \
0  1356998405000000    27.0          0  2012-12-31              3883       RD   
1  1356998407000000    40.0          0  2012-12-31              2226       HN   
2  1356998407000000     7.0          0  2012-12-31              6106       D1   
3  1356998411000000   747.0          0  2012-12-31              3531       SL   
4  1356998411000000    56.0          0  2012-12-31              1830       RD   

   Congestion  LonWGS84   LatWGS84  Delay  BlockID  VehicleID  AtStop  \
0           0 -6.233417  53.342232      0    27017      33521       0   
1           0 -6.278250  53.416683      0    40206      33142       0   
2           0 -6.231633  53.317768      0     7019      43004       1   
3           0 -6.254617  53.355484   -454   747007      40039       0   
4           0 -6.233183  53.342201      0    56001      33488       0   

             datetime      Time  Day  Hour  Minute  
0 2013-01-01 00:00:05

In [9]:
print ('The data has {} Rows and {} columns'.format(df.shape[0],df.shape[1]))
print("The types of columns are:")
df.dtypes

The data has 44455102 Rows and 18 columns
The types of columns are:


Timestamp                    int64
LineID                     float64
Direction                    int64
TimeFrame                   object
VehicleJourneyID             int64
Operator                    object
Congestion                   int64
LonWGS84                   float64
LatWGS84                   float64
Delay                        int64
BlockID                      int64
VehicleID                    int64
AtStop                       int64
datetime            datetime64[ns]
Time                        object
Day                          int64
Hour                         int64
Minute                       int64
dtype: object

In [10]:
# Getting the list of column Types
CategoricalColumns = list(set(['LineID', 'Direction', 'TimeFrame', 'VehicleJourneyID', 'Operator', 'Congestion',  'Delay', 'BlockID', 'VehicleID', 'StopID', 'AtStop']))
NumericalColumns = list(set(list(df.select_dtypes([np.number]).columns)))
#objectTypeColumns = list(set(list(df.select_dtypes([np.object_]).columns)))
DateColumns = list(set(list(df.select_dtypes([np.datetime64]).columns)))


#print(objectTypeColumns)


CategoricalColumns = list(set(CategoricalColumns) - set(ColumnsToExclude)-set(DateColumns))
NumericalColumns = list(set(NumericalColumns)-set(ColumnsToExclude)-set(DateColumns))

print(CategoricalColumns)
print(NumericalColumns)
print(DateColumns)


['LineID', 'BlockID', 'AtStop', 'Operator', 'Congestion', 'TimeFrame', 'Direction', 'Delay', 'VehicleJourneyID', 'VehicleID']
['LineID', 'Timestamp', 'Minute', 'LatWGS84', 'BlockID', 'AtStop', 'Congestion', 'Day', 'Direction', 'Delay', 'VehicleJourneyID', 'LonWGS84', 'Hour', 'VehicleID']
['datetime']


In [11]:
def num_missing(x):
    return len(x.index)-x.count()

def num_unique(x):
    return len(np.unique(x))

temp_df = df.describe().T
missing_df = pd.DataFrame(df.apply(num_missing, axis=0)) 
missing_df.columns = ['missing']  # type: ignore
unq_df = pd.DataFrame(df.apply(num_unique, axis=0))
unq_df.columns = ['unique']  # type: ignore
types_df = pd.DataFrame(df.dtypes)
types_df.columns = ['DataType'] # type: ignore

In [12]:
df.describe()


,Timestamp,LineID,Direction,VehicleJourneyID,Congestion,LonWGS84,LatWGS84,Delay,BlockID,VehicleID,AtStop,Day,Hour,Minute
count,4.445510e+07,4.445304e+07,44455102.0,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07,4.445510e+07
mean,1.358318e+15,7.796879e+01,0.0,9.417122e+03,1.163261e-02,-6.272803e+00,5.334513e+01,-2.816513e+01,1.092438e+05,3.542925e+04,2.335773e-01,1.567200e+01,1.388824e+01,2.946709e+01
std,7.394928e+11,1.146419e+02,0.0,6.159844e+04,1.072254e-01,8.389025e-02,5.488354e-02,4.729678e+02,1.921244e+05,3.281093e+03,4.231063e-01,8.563509e+00,4.921598e+00,1.729008e+01
min,1.356998e+15,1.000000e+00,0.0,1.000000e+00,0.000000e+00,-6.617517e+00,5.306802e+01,-1.504500e+04,3.900000e+02,2.804700e+04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.357679e+15,2.500000e+01,0.0,2.536000e+03,0.000000e+00,-6.308650e+00,5.332005e+01,-2.090000e+02,1.602000e+04,3.330800e+04,0.000000e+00,8.000000e+00,1.000000e+01,1.500000e+01
50%,1.358327e+15,4.000000e+01,0.0,4.718000e+03,0.000000e+00,-6.261600e+00,5.334645e+01,0.000000e+00,4.020500e+04,3.352500e+04,0.000000e+00,1.600000e+01,1.400000e+01,2.900000e+01
75%,1.358955e+15,8.300000e+01,0.0,6.769000e+03,0.000000e+00,-6.233166e+00,5.337533e+01,1.040000e+02,8.400400e+04,3.802500e+04,0.000000e+00,2.300000e+01,1.800000e+01,4.400000e+01
max,1.359633e+15,7.470000e+02,0.0,9.998560e+05,1.000000e+00,-6.052917e+00,5.360873e+01,1.161220e+05,8.350020e+05,4.307800e+04,1.000000e+00,3.100000e+01,2.300000e+01,5.900000e+01


In [13]:
summary_df = temp_df.join(missing_df).join(unq_df).join(types_df)
print(summary_df)

                       count          mean           std           min  \
Timestamp         44455102.0  1.358318e+15  7.394928e+11  1.356998e+15   
LineID            44453044.0  7.796879e+01  1.146419e+02  1.000000e+00   
Direction         44455102.0  0.000000e+00  0.000000e+00  0.000000e+00   
VehicleJourneyID  44455102.0  9.417122e+03  6.159844e+04  1.000000e+00   
Congestion        44455102.0  1.163261e-02  1.072254e-01  0.000000e+00   
LonWGS84          44455102.0 -6.272803e+00  8.389025e-02 -6.617517e+00   
LatWGS84          44455102.0  5.334513e+01  5.488354e-02  5.306802e+01   
Delay             44455102.0 -2.816513e+01  4.729678e+02 -1.504500e+04   
BlockID           44455102.0  1.092438e+05  1.921244e+05  3.900000e+02   
VehicleID         44455102.0  3.542925e+04  3.281093e+03  2.804700e+04   
AtStop            44455102.0  2.335773e-01  4.231063e-01  0.000000e+00   
Day               44455102.0  1.567200e+01  8.563509e+00  1.000000e+00   
Hour              44455102.0  1.388824

In [14]:
col_names = list(types_df.index)
num_cols = len(col_names)
index = range(num_cols)
cat_index = []
for i in index:
    if col_names[i] in CategoricalColumns:
        cat_index.append(i)
summary_df_cat = missing_df.join(unq_df).join(types_df.iloc[cat_index], how='inner') #Only summarize categorical columns
print(summary_df_cat)

                  missing  unique DataType
LineID               2058      67  float64
Direction               0       1    int64
TimeFrame               0      32   object
VehicleJourneyID        0   18614    int64
Operator                0       8   object
Congestion              0       2    int64
Delay                   0   12638    int64
BlockID                 0    1238    int64
VehicleID               0     911    int64
AtStop                  0       2    int64


In [ ]:
df.to_parquet('./Data/Bus/CleanedBusData.parquet')

Download the weather station details https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv

In [15]:
url = 'https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv'


Load the weather statio details for Dublin

In [16]:
weatherStations = pd.read_csv(url)
weatherStations = weatherStations[weatherStations['county'].str.contains('Dublin')]
weatherStations.head(10)

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
575,Dublin,10323,STILLORGAN (FARMLEIGH),67,320514,227230,53.28194,-6.19389,1915,1935
576,Dublin,10123,DUBLIN (FITZWILLIAM SQUARE),16,316404,233098,53.33556,-6.25333,1908,1937
577,Dublin,10223,ROYAL BOTANIC GARDENS (GLASNEVIN,17,315083,237179,53.37250,-6.27167,1880,1925
578,Dublin,1223,BELGARD CASTLE,125,307400,229300,53.30278,-6.38889,1914,1963
579,Dublin,632,LUSK G.S.,23,321700,254300,53.52500,-6.16528,1949,2012
580,Dublin,1232,KINSALEY (AGR.RES.STN.),19,321500,242900,53.42222,-6.17222,1961,2004
581,Dublin,2523,DUBLIN (Ringsend),7,318900,233900,53.34167,-6.21389,1911,(null)
582,Dublin,2623,HOWTH CASTLE,11,327900,239000,53.38611,-6.07778,1913,1971
583,Dublin,2723,STILLORGAN (VARTRY HSE.),80,320200,226800,53.27778,-6.19722,1915,1999
584,Dublin,2732,ARDGILLAN,62,321791,261102,53.58583,-6.16167,1894,1938


In [ ]:

#sns.pairplot(df.sample(frac=0.3, replace=False))

In [17]:
dublinStations = weatherStations[["station name", "name"]]

temp_df = dublinStations.describe().T
missing_df = pd.DataFrame(dublinStations.apply(num_missing, axis=0)) 
missing_df.columns = ['missing']  # type: ignore
unq_df = pd.DataFrame(dublinStations.apply(num_unique, axis=0))
unq_df.columns = ['unique']  # type: ignore
types_df = pd.DataFrame(dublinStations.dtypes)
types_df.columns = ['DataType']  # type: ignore
summary_df = temp_df.join(missing_df).join(unq_df).join(types_df)
print(summary_df)

              count         mean          std    min      25%     50%     75%  \
station name   94.0  4237.542553  3086.169009  124.0  1554.75  3473.5  6773.0   

                  max  missing  unique DataType  
station name  10323.0        0      94    int64  


In [18]:
col_names = list(types_df.index)
num_cols = len(col_names)
index = range(num_cols)
cat_index = []
for i in index:
    if col_names[i] in dublinStations[["name"]]:
        cat_index.append(i)
summary_df_cat = missing_df.join(unq_df).join(types_df.iloc[cat_index], how='inner') #Only summarize categorical columns
print(summary_df_cat)  # there are 2 stations named DUBLIN (Glasnevin)

      missing  unique DataType
name        0      93   object


In [19]:
list(types_df.index)

['station name', 'name']

In [20]:
dublinStations.columns = [x.strip().replace(' ', '') for x in dublinStations.columns]
dublinStations.describe

<bound method NDFrame.describe of      stationname                              name
575        10323            STILLORGAN (FARMLEIGH)
576        10123       DUBLIN (FITZWILLIAM SQUARE)
577        10223  ROYAL BOTANIC GARDENS (GLASNEVIN
578         1223                    BELGARD CASTLE
579          632                         LUSK G.S.
..           ...                               ...
664         9623       KNOCKLYON (ST.COLUMCILLE'S)
665         9723                 Dublin-Navan_Road
666          623                           BRITTAS
667         9823           DUBLIN (MERRION SQUARE)
668         2423                 DUBLIN (CLONTARF)

[94 rows x 2 columns]>

In [24]:
for rows in dublinStations.itertuples():
    # get hourly weather data
    url = "http://cli.fusio.net/cli/climate_data/webdata/hly" + str(rows.stationname) + ".zip"
    print(url)
    dest = './Zips/MetEirrean/hly' + str(rows.stationname) + ".zip"
    try:
        weatherFile = rq.urlretrieve(url, dest)
        continue
    except Exception:
        print('No file exists')  
    # get the daily weather data
    url = "http://cli.fusio.net/cli/climate_data/webdata/dly" + str(rows.stationname) + ".zip"
    print(url)
    dest = './Zips/MetEirrean/dly' + str(rows.stationname) + ".zip"
    try:
        weatherFile = rq.urlretrieve(url, dest)
        continue
    except Exception:
        print('No file exists')  

http://cli.fusio.net/cli/climate_data/webdata/hly10323.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/dly10323.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/hly10123.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/dly10123.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/hly10223.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/dly10223.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/hly1223.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/dly1223.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/hly632.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/dly632.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/hly1232.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/dly1232.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/hly2523.zip
No file exists
http://cli.fusio.net/cli/climate_data/webdata/d

In [21]:
os.getcwd()

'n:\\GitHubSource\\DublinBusWeatherProject'

In [27]:
import zipfile as zip
    # Zip creates its own folders - no need to check for folder existence
files = os.listdir('./Zips/MetEirrean/')
WDfiles = [f for f in files if f.startswith('hly')]
for f in WDfiles:
    fp = './Zips/MetEirrean/' + f
    with zip.ZipFile(fp,  mode='r') as arc: 
        arc.extractall('./Data/MetEirrean/')



In [29]:
sdf = pd.read_csv('./Data/MetEirrean/hly3723.csv',  delimiter=',', header=0,  parse_dates=True, low_memory=True,skiprows=9)
    

ParserError: Error tokenizing data. C error: Expected 1 fields in line 24, saw 21


In [30]:
files = os.listdir('./Data/MetEirrean/')  
WDfiles = [f for f in files if f.endswith('.csv')] # need to remove the empty files

In [31]:
def load_csv_Files(direc, files):

    for f in files:
        # need to get number of rows to skip 
        temp=pd.read_csv(direc + f,sep='^',header=None,prefix='X')
        temp2=temp.X0.str.split(',',expand=True)
        del temp['X0']
        temp=pd.concat([temp,temp2],axis=1)
        cols = list(range(0,temp.shape[1]))

        print(f)
        yield pd.read_csv(direc + f,  delimiter=',', header=0,  parse_dates=True, low_memory=True, skiprows=14, usecols=cols, na_values='NAN')
    

In [32]:
wdf = pd.concat(load_csv_Files('./Data/MetEirrean/', WDfiles), copy = False)




C:\Users\stehayes\AppData\Local\Temp\ipykernel_100\925239655.py:5: FutureWarning: The prefix argument has been deprecated and will be removed in a future version. Use a list comprehension on the column names in the future.


  temp=pd.read_csv(direc + f,sep='^',header=None,prefix='X')


hly175.csv


C:\Users\stehayes\AppData\Local\Temp\ipykernel_100\925239655.py:12: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  yield pd.read_csv(direc + f,  delimiter=',', header=0,  parse_dates=True, low_memory=True, skiprows=14, usecols=cols, na_values='NAN')
C:\Users\stehayes\AppData\Local\Temp\ipykernel_100\925239655.py:5: FutureWarning: The prefix argument has been deprecated and will be removed in a future version. Use a list comprehension on the column names in the future.


  temp=pd.read_csv(direc + f,sep='^',header=None,prefix='X')


hly3723.csv


ValueError: Usecols do not match columns, columns expected but not found: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [ ]:
a = list(range(0,9))


In [ ]:
wdf.shape

(249744, 26)